# Predicing Year of Marriage

## End to End Machine Learning Deployment with Flask-AWS

#### PART - 1 : Model Building and hosting local API

* Data Preparation
* Machine Learning Modelling
* Model Evaluation
* Export Trained Model
* LOCAL REST API with Flask web-server
* Create a website for predicing marriage age calling REST API

#### PART - 2 : Deploying Public API to AWS EC2 server and launch website service

* Spin up an EC2 server
* Configure EC2 with security group and private key
* Install libraries and dependencies on the EC2 server
* Move trained model and app.py flask files to EC2 (winscp)
* Configure flaskapp.wsgi file and Apache vhost file
* Restart apache webserver and Check API status
* Launch a website with domain name and host webpage.

In [124]:
import pandas as pd
data = pd.read_csv('age_of_marriage_data.csv')
print(data.shape)
data.head()

(2567, 10)


,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


In [125]:
data.isnull().sum()

id                   0
gender              29
height             118
religion           635
caste              142
mother_tongue      164
profession         330
location           155
country             16
age_of_marriage     19
dtype: int64

In [126]:
data.dropna(axis=0,how='any',subset=['age_of_marriage'],inplace = True)

In [127]:
data.shape

(2548, 10)

In [128]:
data.isnull().sum()

id                   0
gender              29
height             118
religion           616
caste              132
mother_tongue      157
profession         324
location           151
country             16
age_of_marriage      0
dtype: int64

In [135]:
data.dropna(inplace=True)

In [136]:
# data = imputed_X_train

In [152]:
from sklearn.impute import KNNImputer
import pandas as pd

cols_backup = data.columns
data['gender'].fillna('#', inplace=True)  # replace all missing data in desired column with with '#'
imputer = KNNImputer(missing_values='#')  # tell the imputer to consider only '#' as missing data
imputed_data = imputer.fit_transform(data)  # impute all '#'
data = pd.DataFrame(data=imputed_data, columns=cols_backup)

ValueError: could not convert string to float: 'male'

In [137]:
data.shape

(1932, 10)

In [138]:
X = data.loc[:,['gender','height','religion','caste','mother_tongue','country','profession','location']]
#y = data.loc[:,['age_of_marriage']]
y = data.age_of_marriage

In [151]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.loc[:,['gender','religion','caste','mother_tongue','country','profession','location']]= \
X.loc[:,['gender','religion','caste','mother_tongue','country','profession','location']].apply(enc.fit_transform)

In [153]:
X.head()

,gender,religion,caste,mother_tongue,country,profession,location,height_cms
1,1,2,34,6,19,22,153,170.18
2,1,1,14,8,5,25,56,170.18
3,0,1,36,8,5,7,295,152.40
4,1,0,13,13,5,68,410,165.10
5,1,1,38,8,5,74,124,165.10


In [141]:
def h_cms(h):
    return int(h.split('\'')[0])*30.48 + int(h.split('\'')[1].replace('"',''))*2.54

X['height_cms'] = X.height.apply(h_cms)

X.drop('height',inplace=True,axis=1)
X.head()

In [144]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [145]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=80,max_depth=11)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [146]:
from sklearn.metrics import mean_absolute_error, r2_score
print("MAE : ", mean_absolute_error(y_test,y_predict))
r2_score(y_test,y_predict)

MAE :  1.045160008014447


0.6941664248087973